In [1]:
import nbformat

notebook_path = "Context Window Enhancement.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


Fixed!


In [1]:
!pip install langchain-community langchain-text-splitters langchain-ollama pypdf ollama langchain-huggingface sentence-transformers faiss-cpu  --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 80.6 MB/s eta 0:00:00


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from typing import List
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

參數

In [12]:
path="/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
Chunk_size=512
Chunk_overlap=64
query="What the CRAG proposed to improve"

In [4]:
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

標記chunks的位置

In [5]:
def textx_into_chunks_with_indices(path, chunk_size:int, chunk_overlap:int) -> List[Document]:
    loader=PyPDFLoader(path)
    docs=loader.load()
    texts=[doc.page_content for doc in docs]
    text="\n".join(texts)

    chunks=[]
    start=0
    while start < len(text):
        end=start+chunk_size
        chunk=text[start:end]
        chunks.append(
            Document
                (
                    page_content=chunk,
                    metadata={"index":len(chunks) , "text":text}
                )
            )
        start=start+chunk_size-chunk_overlap
    return chunks

docs=textx_into_chunks_with_indices(path,Chunk_size,Chunk_overlap)

創vectorstore和retriever

In [21]:
VectorStore=FAISS.from_documents(docs,embedding_model)
retriever=VectorStore.as_retriever()

def get_chunk_by_index(docs, target_index:int) -> Document:
    for doc in docs:
        if doc.metadata.get('index') == target_index:
            return doc
    return None

In [7]:
chunk = get_chunk_by_index(docs, 0)
print(chunk.page_content)

Corrective Retrieval Augmented Generation
Shi-Qi Yan1*, Jia-Chen Gu2*, Yun Zhu3, Zhen-Hua Ling1
1National Engineering Research Center of Speech and Language Information Processing,
University of Science and Technology of China, Hefei, China
2Department of Computer Science, University of California, Los Angeles
3Google DeepMind
yansiki@mail.ustc.edu.cn, gujc@ucla.edu, yunzhu@google.com, zhling@ustc.edu.cn
Abstract
Large language models (LLMs) inevitably
exhibit hallucinations since the accuracy of
generated 


把檢索的文章與前後相鄰文章拼接

In [15]:
def retrieve_with_context_overlap( retriever, query:str, chunk_size, chunk_overlap, docs, num_neighbors: int = 1,) -> List[str]:
  relevant_chunks=retriever.invoke(query)
  result=[]

  for chunk in relevant_chunks:
    chunk_index=chunk.metadata.get('index')

    start_index=max(0,chunk_index-num_neighbors)
    end_index=chunk_index+num_neighbors+1

    neighbor_chunks = []

    for i in range(start_index, end_index):
        neighbor_chunk = get_chunk_by_index(docs, i)
        if neighbor_chunk:
          neighbor_chunks.append(neighbor_chunk)

    neighbor_chunks.sort(key=lambda x: x.metadata.get('index'))

    concatenated_text = neighbor_chunks[0].page_content
    for i in range(1, len(neighbor_chunks)):
      current_chunk = neighbor_chunks[i].page_content
      overlap_start=max(0 , len(concatenated_text)-chunk_overlap)
      concatenated_text=concatenated_text[ :overlap_start]+current_chunk
    result.append(concatenated_text)
  return result




Comparing regular retrival and retrival with context window

In [20]:

baseline_chunk = retriever.invoke(query
    ,
    k=1
)
# Focused context enrichment approach
enriched_chunks = retrieve_with_context_overlap(
    retriever,
    query,
    num_neighbors=1,
    chunk_size=Chunk_size,
    chunk_overlap=Chunk_overlap,
    docs=docs
)

print("Baseline Chunk:")
print(baseline_chunk[0].page_content)
print("\nEnriched Chunks:")
print(enriched_chunks[0])

Baseline Chunk:
 Corrective Retrieval
Augmented Generation is proposed to improve the
robustness of generation. Essentially, a lightweight
retrieval evaluator is to estimate and trigger three
knowledge retrieval actions discriminately. With
the further leverage of web search and optimized
knowledge utilization, CRAG has significantly
improved the ability of automatic self-correction
and efficient utilization of retrieved documents.
Experiments extensively demonstrate its adaptabil-
ity to RAG-based approaches as well as ge

Enriched Chunks:
xecution time per
instance in practice, as detailed in Table 6. The
findings indicate that the self-correction mecha-
nism incurs only modest computational overhead
while significantly enhancing performance, thereby
validating its lightweight nature.
6 Conclusion & Limitation
This paper studies the problem where RAG-based
approaches are challenged if retrieval goes wrong,
thereby exposing inaccurate and misleading knowl-
edge to generative LMs. Corr